In [1]:
import jax

import jax.numpy as np

from numpy.polynomial.hermite import hermgauss

from structure_vb_lib import structure_model_lib, data_utils
import structure_vb_lib.structure_optimization_lib as s_optim_lib

import paragami

import time

import pprint

/accounts/grad/runjing_liu/.conda/envs/bnp_sensitivity_jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
import numpy as onp
onp.random.seed(53453)

# Simulate data

In [3]:
# this is the size of the human genome diversity data
# n_obs = 1107 # number of individuals
# n_loci = 2810 # number of loci

n_obs = 25 # number of individuals
n_loci = 75 # number of loci

n_pop = 5 # number of true populations

In [4]:
# g_obs is array of size (n_obs x n_loci x 3)
g_obs, true_z, true_pop_allele_freq, true_ind_admix_propn = \
    data_utils.draw_data(n_obs, n_loci, n_pop)

In [5]:
g_obs = np.array(g_obs)

# VB parameters

In [6]:
# number of components in truncated BNP approximation
k_approx = 20

In [7]:
# randomly initialized vb parameters along with corresponding pattern
vb_params_dict, vb_params_paragami = \
    structure_model_lib.get_vb_params_paragami_object(n_obs,
                                                      n_loci,
                                                      k_approx,
                                                      use_logitnormal_sticks = True)
    
print(vb_params_paragami)

OrderedDict:
	[pop_freq_beta_params] = NumericArrayPattern (75, 20, 2) (lb=0.0, ub=inf)
	[ind_admix_params] = OrderedDict:
	[stick_means] = NumericArrayPattern (25, 19) (lb=-inf, ub=inf)
	[stick_infos] = NumericArrayPattern (25, 19) (lb=0.0, ub=inf)


In [8]:
gh_deg = 8
gh_loc, gh_weights = hermgauss(8)

In [13]:
vb_free = vb_params_paragami.flatten(vb_params_dict, free = True)

In [14]:
from structure_vb_lib import posterior_quantities_lib

In [15]:
@jax.jit
def get_n_clusters_insample(vb_free): 
    vb_params_dict = vb_params_paragami.fold(vb_free, free = True)
    
    return posterior_quantities_lib.\
            get_e_num_clusters(g_obs, 
                               vb_free,
                               gh_loc,
                               gh_weights, 
                               threshold = 800,
                               n_samples = 200,
                               seed = 2342, 
                               return_samples = True)

In [17]:
t0 = time.time()
_ = get_n_clusters_insample(vb_free).block_until_ready()
print(time.time() - t0)

NameError: name 'vb_opt_dict' is not defined

In [10]:
structure_posterior_quantities_lib.get_e_num_clusters(g_obs, 
                                       vb_params_dict,
                                       gh_loc,
                                       gh_weights, 
                                       threshold = 0,
                                       n_samples = 10,
                                       seed = 2342)

DeviceArray(8.9, dtype=float64)

In [11]:
stick_means = vb_params_dict['ind_admix_params']['stick_means']
stick_infos = vb_params_dict['ind_admix_params']['stick_infos']

In [12]:
structure_posterior_quantities_lib.get_e_num_pred_clusters(stick_means, stick_infos,
                                                           gh_loc, gh_weights, n_samples = 10)


(10, 25, 19)
(10,)


DeviceArray(3.98616095, dtype=float64)